In [1]:
#%pip install requests
#%pip install bs4

In [1]:
from bs4 import BeautifulSoup # type: ignore
import requests # type: ignore
import pandas as pd # type: ignore
import urllib.parse
#import os
#from PIL import Image
#from IPython.display import IFrame

In [2]:
import numpy as np

In [2]:
#%pip install selenium

In [3]:
from selenium import webdriver # type: ignore
from selenium.webdriver.chrome.options import Options # type: ignore
from selenium.webdriver.chrome.service import Service # type: ignore
from selenium.webdriver.support.ui import WebDriverWait # type: ignore
from selenium.common.exceptions import TimeoutException # type: ignore
from time import sleep

In [3]:
#url_get = 'https://shopee.com.my/search?'
#payload = {"keyword":"dashcam"}
#r = requests.get(url_get, params=payload)
#print(r.url)
#print(r.status_code)
#print(r.content)

In [6]:
#base_url = 'https://shopee.com.my/search?keyword=dashcam&trackingId=searchhint-1728046904-ce774037-8250-11ef-8e22-3ea6d88eabcb'
#base_url = 'https://shopee.com.my/shop/13377506/search?page=0&sortBy=sales'
#base_url = 'https://www.lazada.com.my/tag/dashcam/?spm=a2o4k.homepage.search.d_go&q=dashcam&catalog_redirect_tag=true'
base_url = 'https://www.lazada.com.my/tag/dashcam/?catalog_redirect_tag=true&page=1&q=dashcam&spm=a2o4k.homepage.search.d_go'
chrome_options = Options()

# set chrome driver options to disable any popup's from the website
# to find local path for chrome profile, open chrome browser
# and in the address bar type, "chrome://version"
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('start-maximized')
chrome_options.add_argument('user-data-dir=C:\\Users\\1\\AppData\\Local\\Google\\Chrome\\User Data\\Default')
# To disable the message, "Chrome is being controlled by automated test software"
chrome_options.add_argument("disable-infobars")
# Pass the argument 1 to allow and 2 to block
chrome_options.add_experimental_option("prefs", { 
    "profile.default_content_setting_values.notifications": 2
    })
# invoke the webdriver
chrome_service = Service(executable_path = r"C:\Users\1\OneDrive\Desktop\Personal Matter\Data Analytics Learning\Self Exploration Project\chromedriver-win64\chromedriver.exe")
browser = webdriver.Chrome(service = chrome_service, options = chrome_options)

#browser.get(base_url)

delay = 5 #seconds

In [9]:
item_cost, item_init_cost, item_loc = [],[],[]
item_name, items_sold, discount_percent = [], [], []

for i in range(30):
    page_num = i + 1
    base_url = 'https://www.lazada.com.my/tag/dashcam/?catalog_redirect_tag=true&page=' + str(page_num) + '&q=dashcam&spm=a2o4k.homepage.search.d_go'
    browser.get(base_url)
    
    while True:
        try:
            WebDriverWait(browser, delay)
            print ("Page ", str(page_num), " is ready")
            sleep(5)
            html = browser.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
            #print(html)
            soup = BeautifulSoup(html, "html.parser")

            # find_all() returns an array of elements. 
            # We have to go through all of them and select that one you are need. And than call get_text()
            for item_n in soup.find_all('div', class_='RfADt'):
                #print(item_n.get_text())
                item_name.append(item_n.text)

            # find the price of items
            for item_c in soup.find_all('span', class_='ooOxS'):
                #print(item_c.get_text())
                item_cost.append(item_c.text)

            # find initial item cost
            #for item_ic in soup.find_all('div', class_ = '_1w9jLI QbH7Ig U90Nhh'):
            #    print(item_ic.get_text())
            #    item_init_cost.append(item_ic.text)
            
            # find total number of items sold/month
            for items_s in soup.find_all('span',class_ = '_1cEkb'):
                #print(items_s.get_text())
                items_sold.append(items_s.text)

            # find item discount percent
            for dp in soup.find_all('span', class_ = 'ic-dynamic-badge ic-dynamic-badge-text ic-dynamic-badge-120015 ic-dynamic-group-2'):
                #print(dp.get_text())
                discount_percent.append(dp.text)

            # find item location
            #for il in soup.find_all('div', class_ = '_3amru2'):
            #    print(il.get_text())
            #    item_loc.append(il.text)

            break # it will break from the loop once the specific element will be present. 
        except TimeoutException:
            print ("Loading took too much time!-Try again")

# close the automated browser
browser.close()

Page  1  is ready
Page  2  is ready
Page  3  is ready
Page  4  is ready
Page  5  is ready
Page  6  is ready
Page  7  is ready
Page  8  is ready
Page  9  is ready
Page  10  is ready
Page  11  is ready
Page  12  is ready
Page  13  is ready
Page  14  is ready
Page  15  is ready
Page  16  is ready
Page  17  is ready
Page  18  is ready
Page  19  is ready
Page  20  is ready
Page  21  is ready
Page  22  is ready
Page  23  is ready
Page  24  is ready
Page  25  is ready
Page  26  is ready
Page  27  is ready
Page  28  is ready
Page  29  is ready
Page  30  is ready


In [10]:
#rows = zip(item_name, item_cost, discount_percent, items_sold)

In [10]:
import csv
newFilePath = 'lazada_item_list.csv'
#with open(newFilePath, "w") as f:
#    writer = csv.writer(f)
#    for row in rows:
#        writer.writerow(row)

In [35]:
df = pd.DataFrame(columns= ['product_name', 'price_RM', 'discount_percent', 'quantity_sold'])
df['product_name'] = pd.Series(item_name)
df['price_RM'] = pd.Series(item_cost)
df['discount_percent'] = pd.Series(discount_percent)
df['quantity_sold'] = pd.Series(items_sold)

df['price_RM'] = df['price_RM'].replace(['RM', ","], '', regex=True)
df['price_RM'] = df['price_RM'].astype(float)

df['discount_percent'] = df['discount_percent'].replace(to_replace=np.nan, value='Voucher save 0%')
df['discount_percent'] = df['discount_percent'].replace(to_replace="Voucher save ", value="", regex=True)
df['discount_percent'] = df['discount_percent'].replace(to_replace="%", value="", regex=True)
df['discount_percent'] = df['discount_percent'].astype(int)

df['original_price_RM'] = df['price_RM'] / (1 - (df['discount_percent'] / 100))
df['original_price_RM'] = df['original_price_RM'].round(2)
df['original_price_RM'] = df['original_price_RM'].astype(float)

df['quantity_sold'] = df['quantity_sold'].replace(' sold', '', regex=True)
df['quantity_sold'] = df['quantity_sold'].replace('K', '000', regex=True)
df['quantity_sold'] = df['quantity_sold'].replace(np.nan, '0')
df['quantity_sold'] = df['quantity_sold'].astype(str).str.replace(".", "")
df['quantity_sold'] = df['quantity_sold'].astype(int)

print(df.dtypes)
df.head()

product_name          object
price_RM             float64
discount_percent       int64
quantity_sold          int64
original_price_RM    float64
dtype: object


,product_name,price_RM,discount_percent,quantity_sold,original_price_RM
0,DOSEN S50 Driving Recorder Front interior Rear...,135.80,50,228,271.60
1,DOSNE A1 Driving Recorder Front interior Rear ...,79.80,43,323,140.00
2,DOSNE Wifi 行车记录仪 24 Hours Record 1080P FHD wif...,79.80,43,341,140.00
3,DOSEN A1 wireless Driving Recorder Front inter...,78.49,61,31000,201.26
4,70mai Dashcam A200 Dual Channel 1080P Full HD+...,249.00,19,16000,307.41


In [36]:
df.to_csv(newFilePath)